In [1]:
import pandas as pd
import json
import re
import shutil
import os
import unidecode

In [2]:

def evolVal(valI, valE):
    if (valE == 0):
        return 0
    elif(valI != valE):
        return valI - valE
    else:
        return 0

def evolPercent(ev, val):
    if val != 0: 
        return round((ev / val)*100,2) 
    else: 
        return None

def get_last_data(dff):
    most_recent_date = dff['period_date'].max()
    dfDate = dff.loc[dff.period_date == most_recent_date]
    return [most_recent_date, dfDate["valeur"].sum()]
    
def get_evolution(dff, last_date, last_value):
    dfevol = pd.DataFrame()
    dfevol = dff.copy() 
    dfevol.drop(dfevol.loc[dfevol['period_date'] == last_date].index, inplace=True)
    previous_last_data = get_last_data(dfevol)
    evol = evolVal(last_value, previous_last_data[1])
    evol_percent = evolPercent(evol, previous_last_data[1])
    return [evol, evol_percent]

def get_data_history(dff):
    dates = dff.sort_values(by="period_date").period_date.unique()
    #print(dates)
    values = []
    for date in dates:
        value = dff.loc[dff.period_date == date]
        values += [{"date": date.astype(str), "value": value["valeur"].sum()}]
        
    return values 

def get_level(df, level, code_level):
    last_data = get_last_data(df)
    evolution = get_evolution(df, last_data[0], last_data[1])
    data_history = get_data_history(df)
    data_level = {}
    data_level["level"] = level
    data_level["code_level"] = code_level
    data_level["last_value"] = last_data[1]
    data_level["last_date"] = str(last_data[0]) #.to_pydatetime()
    data_level["evol"] = evolution[0]
    data_level["evol_percentage"] = evolution[1]
    data_level["evol_color"] = "red"
    data_level["values"] = data_history
    return data_level

def clean_folder(folder):
    # Suppression des fichiers générés précédemment
    try:
        shutil.rmtree(folder)
    except:
        None
    os.makedirs(folder)

def write_file(data, file_name, folder):
    newfilename = file_name

    newfilename = re.sub(' ', '_', newfilename)
    newfilename = re.sub('/', '&', newfilename)
    newfilename = re.sub('"', '', newfilename)
    newfilename = re.sub('’', '_', newfilename)
    newfilename = unidecode.unidecode(newfilename)

    path = folder + "/" + newfilename +".json"
    print(newfilename)
    with open(path, "w", encoding="utf8") as f:
        json.dump(data, f, ensure_ascii=False)


def clean_file(file_name):
    # Suppression du fichier généré précédemment
    try:
        os.remove(file_name)
    except:
        None
    
def append_to_file(data, file_name):
    with open(file_name, "a", encoding="utf8") as output_file:
        json.dump(data, output_file, ensure_ascii=False)
        output_file.write('\n')

In [3]:
col = ["indicateur", "period_date", "valeur", "mesure", "short_indic", "dep", "reg"]
df = pd.read_csv("./pp_dep.csv", usecols=col, sep=";")
df['period_date'] = pd.to_datetime(df['period_date'])

In [4]:
file_name = './france-relance-data-tableau-de-bord.txt'
clean_file(file_name)

indicateurs = df.indicateur.unique()
#print (indicateurs)
for indicateur in indicateurs:
    df_indicateur = df.loc[df.indicateur == indicateur]
    
    print(indicateur)
    #print(df_indicateur.sort_values(by="period_date").period_date.unique().size)
    
    data = {}
    data["nom"] = df_indicateur["indicateur"].iloc[0]
    data["unite"] = df_indicateur["short_indic"].iloc[0]
    # France
    france = get_level(df_indicateur, "nat", "fra")
    
    c = []
    c.append(france)
    data["france"] = c
    
    # Régions
    regions_data = []
    regions = df_indicateur.reg.unique()
    for region in regions:
        df_indicateur_region = df_indicateur.loc[df.reg == region]
        region_data = get_level(df_indicateur_region, "reg", region)
    
        regions_data.append(region_data)
        #print (data)
    
    data["regions"] = regions_data
    #del regions
    #del df_indicateur_region
    
    # Départements
    departements_data = []
    departements = df_indicateur.dep.unique()
    for departement in departements:
        df_indicateur_departement = df_indicateur.loc[df.dep == departement]
        departement_data = get_level(df_indicateur_departement, "dep", departement)
    
        departements_data.append(departement_data)
        #print (data)
    
    data["departements"] = departements_data
    #del departements
    #del df_indicateur_departement
    
    #print (data)
    #write_file(data, data["nom"], "./json")
    append_to_file(data, file_name)
    #del data
    

Nombre de locaux commerciaux connectables à la fibre - THD1
DRI : Nombre d'écoles, collèges et lycées qui feront l'objet de rénovations thermiques - RBC2
DRI : Nombre de projets de rénovation thermique des collectivités soutenus - RBC1
Nombre de conseillers du numérique - INU1
Nombre de communes inscrites au programme - PRR1
Nombre de ponts des collectivités recensés en vue d'un diagnostic - PRR2
Montant de l’investissement total déclenché -  ETE1
Nombre d’entreprises bénéficiaires - ETE2
Emissions de gaz à effet de serre évitées sur la durée de vie des équipements - CBC1
Montant cumulé de l’investissement total ainsi déclenché - CBC2
Nombre d’entreprises ayant reçu l’aide - CBC3
Montant total de CO2 évité - ETH1
Nombre de projets soutenus - ETH2
Quantité de matières plastiques évitées ou dont le recyclage ou l’intégration a été soutenue - IRR1
Nombre de logements subventionnés - RER1
Nombre de projets soutenus - REA1
Nombre d’entreprises ayant reçu l’aide - EEI1
Nombre de PME/TPE - EE

In [5]:
print("Génération des données terminée\n")

Génération des données terminée

